In [1]:
from veggideas.load_data import load_train_data, load_val_data
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
import tensorflow as tf

train_data = load_train_data()
val_data = load_val_data()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading test data
Testing data successfully loaded ✅
Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [2]:
def load_model():
    
    
    model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
    return model

model_transfer = load_model()
model_transfer.summary()

58889256/58889256 [==============================] - 16s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                             

In [3]:
def set_nontrainable_layers(model):
    
    model.trainable = False
    return model

model_transfer = set_nontrainable_layers(model_transfer)
model_transfer.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [56]:
def add_last_layers_try(model):
    base_model = set_nontrainable_layers(model)
    flattening_layer = layers.Flatten()
    prediction_layer = layers.Dense(15, activation='softmax')
    model = models.Sequential([
        base_model,
        flattening_layer,
        prediction_layer
        ])
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

model_transfer = add_last_layers_try(model_transfer)


    

In [4]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])
    
    data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    ])
     
    base_model = set_nontrainable_layers(model)
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(15, activation='relu')
    prediction_layer = layers.Dense(15, activation='softmax')
    
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        flattening_layer,
        dense_layer,
        prediction_layer
        ])
    
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

model_transfer = add_last_layers(model_transfer)


In [5]:
model_transfer.fit(train_data, batch_size=32, epochs=1)


2023-06-06 16:56:12.621553: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 986s 2s/step - loss: 1.9507 - accuracy: 0.4195
